In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from datetime import datetime, timedelta



Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1612373415994_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:

def get_moon_phase(dyn_rec):
    """
    determine the moon phase at a given date
    (mildly off on moon's light intensity)
    checked with http://aa.usno.navy.mil/data/docs/RS_OneDay.html
    
    Returns:

    """
    
    day = dyn_rec['local_time'].day
    year = dyn_rec['local_time'].year
    month = dyn_rec['local_time'].month
    
    ages = [18, 0, 11, 22, 3, 14, 25, 6, 17, 28, 9, 20, 1, 12, 23, 4, 15, 26, 7]
    offsets = [-1, 1, 0, 1, 2, 3, 4, 5, 7, 7, 9, 9]
    description = ["new (totally dark)",
      "waxing crescent (increasing to full)",
      "in its first quarter (increasing to full)",
      "waxing gibbous (increasing to full)",
      "full (full light)",
      "waning gibbous (decreasing from full)",
      "in its last quarter (decreasing from full)",
      "waning crescent (decreasing from full)"]
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
              "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    
    if day == 31:
        day = 1
    days_into_phase = ((ages[(year + 1) % 19] +
                        ((day + offsets[month-1]) % 30) +
                        (year < 1900)) % 30)
    index = int((days_into_phase + 2) * 16/59.0)
    #print(index)  # test
    if index > 7:
        index = 7
    status = description[index]

    # light should be 100% 15 days into phase
    light = int(2 * days_into_phase * 100/29)
    if light > 100:
        light = abs(light - 200);
    date = "%d%s%d" % (day, months[month-1], year)

    dyn_rec['moon_phase'] = status
    return dyn_rec

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
## @params: [TempDir, JOB_NAME]
#args = getResolvedOptions(sys.argv, ['TempDir','JOB_NAME'])

glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session
#job = Job(glueContext)
#job.init(args['JOB_NAME'], args)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
## @type: DataSource
## @args: [database = "historical-weather-db", table_name = "owmhistoricalweather", transformation_ctx = "DataSource0"]
## @return: DataSource0
## @inputs: []
DataSource0 = glueContext.create_dynamic_frame.from_catalog(
    database = "historical-weather-db", 
    table_name = "owmhistoricalweather", 
    transformation_ctx = "DataSource0")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
DataSource0.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- dt: long
|-- dt_iso: string
|-- timezone: long
|-- city_name: string
|-- lat: double
|-- lon: double
|-- temp: double
|-- feels_like: double
|-- temp_min: double
|-- temp_max: double
|-- pressure: long
|-- sea_level: string
|-- grnd_level: string
|-- humidity: long
|-- wind_speed: double
|-- wind_deg: long
|-- rain_3h: string
|-- snow_1h: string
|-- snow_3h: string
|-- clouds_all: long
|-- weather_id: long
|-- weather_main: string
|-- weather_description: string
|-- weather_icon: string
|-- rain_1h: double

In [5]:
print("Source Record Count: " + str(DataSource0.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Source Record Count: 8491

In [6]:
ds0 = DataSource0.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
ds0.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dt: long (nullable = true)
 |-- dt_iso: string (nullable = true)
 |-- timezone: long (nullable = true)
 |-- city_name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- temp: double (nullable = true)
 |-- feels_like: double (nullable = true)
 |-- temp_min: double (nullable = true)
 |-- temp_max: double (nullable = true)
 |-- pressure: long (nullable = true)
 |-- sea_level: string (nullable = true)
 |-- grnd_level: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_deg: long (nullable = true)
 |-- rain_3h: string (nullable = true)
 |-- snow_1h: string (nullable = true)
 |-- snow_3h: string (nullable = true)
 |-- clouds_all: long (nullable = true)
 |-- weather_id: long (nullable = true)
 |-- weather_main: string (nullable = true)
 |-- weather_description: string (nullable = true)
 |-- weather_icon: string (nullable = true)
 |-- rain_1h: double (nullable = true)

In [8]:
from pyspark.sql.functions import to_timestamp, col

ds1 = ds0.withColumn('utc_timestamp', to_timestamp(col('dt_iso'),"yyyy-MM-dd HH:mm:ss z Z"))
ds1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------+---------+---------+----------+-----+----------+--------+--------+--------+---------+----------+--------+----------+--------+-------+-------+-------+----------+----------+------------+-------------------+------------+-------+-------------------+
|        dt|              dt_iso|timezone|city_name|      lat|       lon| temp|feels_like|temp_min|temp_max|pressure|sea_level|grnd_level|humidity|wind_speed|wind_deg|rain_3h|snow_1h|snow_3h|clouds_all|weather_id|weather_main|weather_description|weather_icon|rain_1h|      utc_timestamp|
+----------+--------------------+--------+---------+---------+----------+-----+----------+--------+--------+--------+---------+----------+--------+----------+--------+-------+-------+-------+----------+----------+------------+-------------------+------------+-------+-------------------+
|1577836800|2020-01-01 00:00:...|  -21600| Corrigan|30.996856|-94.827153|48.67|     45.23|    44.6|   54.22|    1019|         |         

In [12]:
from pyspark.sql.functions import expr
ds2 = ds1.withColumn('local_time', ds1.utc_timestamp + expr("INTERVAL -21600 seconds"))
ds2.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------+---------+---------+----------+-----+----------+--------+--------+--------+---------+----------+--------+----------+--------+-------+-------+-------+----------+----------+------------+-------------------+------------+-------+-------------------+-------------------+
|        dt|              dt_iso|timezone|city_name|      lat|       lon| temp|feels_like|temp_min|temp_max|pressure|sea_level|grnd_level|humidity|wind_speed|wind_deg|rain_3h|snow_1h|snow_3h|clouds_all|weather_id|weather_main|weather_description|weather_icon|rain_1h|      utc_timestamp|         local_time|
+----------+--------------------+--------+---------+---------+----------+-----+----------+--------+--------+--------+---------+----------+--------+----------+--------+-------+-------+-------+----------+----------+------------+-------------------+------------+-------+-------------------+-------------------+
|1577836800|2020-01-01 00:00:...|  -21600| Corrigan|30.996856|-94.827153|48.

In [14]:
from awsglue.dynamicframe import DynamicFrame

new_source = DynamicFrame.fromDF(ds2, glueContext , "new_source")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
error_info=''
mooDF = Map.apply(frame = new_source, f= get_moon_phase, info=error_info)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
mooDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- snow_3h: string
|-- lat: double
|-- temp_max: double
|-- timezone: long
|-- sea_level: string
|-- feels_like: double
|-- weather_id: long
|-- utc_timestamp: timestamp
|-- moon_phase: string
|-- temp_min: double
|-- dt: long
|-- wind_deg: long
|-- local_time: timestamp
|-- weather_main: string
|-- clouds_all: long
|-- temp: double
|-- city_name: string
|-- grnd_level: string
|-- pressure: long
|-- weather_icon: string
|-- rain_1h: double
|-- humidity: long
|-- dt_iso: string
|-- snow_1h: string
|-- weather_description: string
|-- rain_3h: string
|-- lon: double
|-- wind_speed: double

In [20]:
mooDF.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+--------+--------+---------+----------+----------+-------------------+--------------------+--------+----------+--------+-------------------+------------+----------+-----+---------+----------+--------+------------+-------+--------+--------------------+-------+-------------------+-------+----------+----------+
|snow_3h|      lat|temp_max|timezone|sea_level|feels_like|weather_id|      utc_timestamp|          moon_phase|temp_min|        dt|wind_deg|         local_time|weather_main|clouds_all| temp|city_name|grnd_level|pressure|weather_icon|rain_1h|humidity|              dt_iso|snow_1h|weather_description|rain_3h|       lon|wind_speed|
+-------+---------+--------+--------+---------+----------+----------+-------------------+--------------------+--------+----------+--------+-------------------+------------+----------+-----+---------+----------+--------+------------+-------+--------+--------------------+-------+-------------------+-------+----------+----------+
|       |30.9

In [21]:
#map fields from source to destination. Fields not mapped are dropped.
## @type: ApplyMapping
## @args: [mappings = [("lon", "double", "lon", "decimal"), ("pressure", "long", "pressure", "decimal"), ("timezone", "long", "tz_offset_sec", "int"), ("city_name", "string", "city_name", "string"), ("dt_iso", "string", "utc_time", "timestamp"), ("temp", "double", "temperature", "decimal"), ("humidity", "long", "rel_humidity", "short"), ("wind_speed", "double", "windspeed", "decimal"), ("lat", "double", "lat", "decimal")], transformation_ctx = "Transform0"]
## @return: Transform0
## @inputs: [frame = DataSource0]
Transform0 = ApplyMapping.apply(
    frame = mooDF, 
    mappings = [
        ("lon", "double", "lon", "decimal"), 
        ("pressure", "long", "pressure", "decimal"), 
        ("timezone", "long", "tz_offset_sec", "int"), 
        ("city_name", "string", "city_name", "string"), 
        ("dt_iso", "string", "utc_time", "timestamp"), 
        ("temp", "double", "temperature", "decimal"), 
        ("humidity", "long", "rel_humidity", "short"), 
        ("wind_speed", "double", "windspeed", "decimal"), 
        ("lat", "double", "lat", "decimal"),
        ("moon_phase", "string", "moon_phase", "string"),
        ("local_time", "timestamp", "local_time", "timestamp")
    ], 
    transformation_ctx = "Transform0")

Transform0.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- lon: decimal
|-- pressure: decimal
|-- tz_offset_sec: int
|-- city_name: string
|-- utc_time: timestamp
|-- temperature: decimal
|-- rel_humidity: short
|-- windspeed: decimal
|-- lat: decimal
|-- moon_phase: string
|-- local_time: timestamp

In [22]:
print("Transformed Record Count: " + str(Transform0.count()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Transformed Record Count: 8491

In [ ]:


## @type: DataSink
## @args: [database = "historical-weather-db", redshift_tmp_dir = TempDir, table_name = "gamepics_public_weather", transformation_ctx = "DataSink0"]
## @return: DataSink0
## @inputs: [frame = Transform0]
DataSink0 = glueContext.write_dynamic_frame.from_catalog(
    frame = Transform0, 
    database = "historical-weather-db", 
    redshift_tmp_dir = args["TempDir"], 
    table_name = "gamepics_public_weather", 
    transformation_ctx = "DataSink0")
job.commit()